# Agent Deployment and Security Workflow (Cloud Run & Agent Engine)

This notebook demonstrates deploying an AI Agent to both Google Cloud Run and Vertex AI Agent Engine, focusing on the integration of **Model Armor** for responsible AI/security filtering. It also illustrates a common deployment issue related to Google Cloud Storage (GCS) permissions on the Agent Engine's custom code service account and how to resolve it.

In [1]:
# 1. Setup and Installation
# This cell installs all the necessary Python libraries for Google Cloud services,
# including Vertex AI, Model Armor, and the Google Agent Development Kit (ADK).

!pip install pyyaml
!pip install cloudpickle==3.1.1
!pip install google-cloud-api-keys
!pip install uvicorn
!pip install fastapi
!pip install python-dotenv
!pip install google-adk
!pip install --upgrade google-cloud-aiplatform
!pip install --upgrade google-cloud-modelarmor
!pip install google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.31.0
    Uninstalling google-genai-1.31.0:
      Successfully uninstalled google-genai-1.31.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.114.0
    Uninstalling google-cloud-aiplatform-1.114.0:
      Successfully uninstalled google-cloud-aiplatform-1.114.0


# 2. Configuration Variables

Set your Google Cloud project details and deployment locations. The `BUCKET` variable is crucial as it serves as the staging location for the Agent Engine deployment, and its permissions will be tested later.

In [2]:
# Update the variable values below with your specific project details.

PROJECT_ID = "data-vpc-sc-demo"  # @param {type:"string"}
PROJECT_NUMBER = "1083677030545"  # @param {type:"string"} # Get this from Project settings in GCP Console
LOCATION = "us-central1" # @param {type:"string"}

# BUCKET is used for storing the agent code package during Agent Engine deployment.
# IMPORTANT: This bucket MUST exist in your project and be in the same region as the deployment.
# We will use a dynamically generated bucket name based on the PROJECT_ID.
BUCKET_NAME = f"{PROJECT_ID}-agent-artifacts"
BUCKET = f"gs://{BUCKET_NAME}"

# Model Armor Template ID (assuming you created this manually)
MODEL_ARMOR_TEMPLATE_ID = "glean-poc-template" # @param {type:"string"}

print(f"Project ID: {PROJECT_ID}")
print(f"Deployment Location: {LOCATION}")
print(f"Agent Engine Staging Bucket: {BUCKET}")

# Ensure the bucket exists (optional, but good practice)
!gsutil mb -p {PROJECT_ID} -l {LOCATION} {BUCKET}

Project ID: data-vpc-sc-demo
Deployment Location: us-central1
Agent Engine Staging Bucket: gs://data-vpc-sc-demo-agent-artifacts
Creating gs://data-vpc-sc-demo-agent-artifacts/...


# 3. Authentication

Authenticate your user session. This is required to execute Google Cloud commands and deploy resources.

In [3]:
from google.colab import auth

# Authenticate the user to access Google Cloud services
auth.authenticate_user(project_id=PROJECT_ID)

# Set the project for subsequent gcloud commands
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


# 4. Model Armor and DLP Setup (Prerequisites)

Model Armor requires its API to be enabled and a template to be created in the GCP Console to define security and responsible AI policies (like DLP and harmful content filtering).

### 4.1 Enable Model Armor API

In [4]:
# Enable the Model Armor API
!gcloud services enable modelarmor.googleapis.com --project={PROJECT_ID}

### 4.2 Manual Step: Create Model Armor Template and DLP Configuration

**Note:** The following steps are typically performed manually in the Google Cloud Console or via the API/gcloud before running this notebook.

1.  **Create Sensitive Data Protection (DLP) Inspection Template** (e.g., ID: `sensitive_data_protection`) in the DLP console.
2.  **Create Model Armor Template** (ID: `glean-poc-template`) in the Model Armor console, linking the DLP template and setting Responsible AI confidence thresholds.

# 5. Stage 1: Cloud Run Deployment

This section clones a sample agent repository, configures the environment variables (including the Model Armor template ID), and deploys the agent as a containerized service using Cloud Run.

In [8]:
CLOUD_RUN_SERVICE_NAME = "capital-agent-service"

# Clone Repository and Prepare Environment
%cd /content/
!rm -rf ai-agent
!git clone https://github.com/avnit/ai-agent.git
%cd ai-agent/

# Create .env file with Model Armor settings for the agent to use
!echo "GOOGLE_CLOUD_PROJECT={PROJECT_ID}" > modelarmor/.env
!echo "GOOGLE_CLOUD_LOCATION={LOCATION}" >> modelarmor/.env
!echo "AIP_ENDPOINT_ID={MODEL_ARMOR_TEMPLATE_ID}" >> modelarmor/.env
!echo "GOOGLE_GENAI_USE_VERTEXAI=true" >> modelarmor/.env

# Deploy to Cloud Run
!gcloud run deploy {CLOUD_RUN_SERVICE_NAME} \
--source . \
--region {LOCATION} \
--project {PROJECT_ID} \
--allow-unauthenticated \
--set-env-vars="GOOGLE_CLOUD_PROJECT={PROJECT_ID},GOOGLE_CLOUD_LOCATION={LOCATION},AIP_ENDPOINT_ID={MODEL_ARMOR_TEMPLATE_ID},GOOGLE_GENAI_USE_VERTEXAI=true"

/content
Cloning into 'ai-agent'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 132 (delta 58), reused 99 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (132/132), 1.67 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/ai-agent
Building using Dockerfile and deploying container to Cloud Run service [capital-agent-service] in project [data-vpc-sc-demo] region [us-central1]
  Setting IAM policy failed, try "gcloud beta run services add-iam-policy-binding --region=us-central1 --member=allUsers --role=roles/run.invoker capital-agent-service"
Service [capital-agent-service] revision [capital-agent-service-00010-gg9] has been deployed and is serving 100 percent of traffic.
Service URL: https://capital-agent-service-1083677030545.us-central1.run.app


# 6. Stage 2: Agent Engine Deployment (Permission Denied Scenario)

Vertex AI Agent Engine deployment requires the Vertex AI Custom Code Service Agent to have read/write access to the staging GCS bucket. We will first identify this service account and then simulate a deployment *failure* due to missing permissions.

In [6]:
# Identify the Vertex AI Custom Code Service Account
# This service account requires permissions on the GCS bucket.
SERVICE_AGENT = f"service-{PROJECT_NUMBER}@gcp-sa-aiplatform-customcode.iam.gserviceaccount.com"
print(f"Vertex AI Custom Code Service Account: {SERVICE_AGENT}")

# Clone and Prepare Agent Engine Repository
%cd /content/
!rm -rf ai-agent-deploy-ae
!git clone https://github.com/avnit/ai-agent-deploy-ae/
%cd ai-agent-deploy-ae/labs/AgentEngineDeploy/

# Configure .env for Agent Engine
!echo "GOOGLE_CLOUD_PROJECT={PROJECT_ID}" > .env
!echo "GOOGLE_CLOUD_LOCATION={LOCATION}" >> .env
!echo "AIP_ENDPOINT_ID={MODEL_ARMOR_TEMPLATE_ID}" >> .env
!echo "GOOGLE_GENAI_USE_VERTEXAI=true" >> .env

# --- ATTEMPT DEPLOYMENT (EXPECTED TO FAIL) ---
# The staging bucket permissions for the Service Agent are currently insufficient.

AGENT_DISPLAY_NAME_FAIL = "adk-agent-test-fail"
print("\n--- ATTEMPTING DEPLOYMENT (EXPECTED TO FAIL: 403 Permission Denied) ---")

# The adk deploy command will fail if the service account does not have Storage Object Admin access to the bucket.
!adk deploy agent_engine \
    --project={PROJECT_ID} \
    --region={LOCATION} \
    --staging_bucket={BUCKET} \
    --display_name={AGENT_DISPLAY_NAME_FAIL} \
    .

# The expected output from the `adk deploy` command would be similar to:
# Deploy failed: 403 GET https://storage.googleapis.com/storage/v1/b/... Permission 'storage.buckets.get' denied...

Vertex AI Custom Code Service Account: service-1083677030545@gcp-sa-aiplatform-customcode.iam.gserviceaccount.com
/content
Cloning into 'ai-agent-deploy-ae'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 81 (delta 32), reused 61 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 28.50 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/ai-agent-deploy-ae/labs/AgentEngineDeploy

--- ATTEMPTING DEPLOYMENT (EXPECTED TO FAIL: 403 Permission Denied) ---
Copying agent source code...
Copying agent source code complete.
Initializing Vertex AI...
Resolving files and dependencies...
Reading environment variables from /content/ai-agent-deploy-ae/labs/AgentEngineDeploy/.env
Ignoring GOOGLE_CLOUD_PROJECT in .env as `--project` was explicitly passed and takes precedence
Ignoring GOOGLE_CLOUD_LOCATION in .env as `--region` was explicitly passed and takes preceden

# 7. Stage 3: Fix Permissions and Successful Redeployment

We will now use the `gcloud storage` command to grant the necessary `Storage Object Admin` role (`roles/storage.objectAdmin`) to the Service Agent on the GCS bucket. Once permissions are granted, the redeployment should succeed.

In [9]:
# --- FIXING PERMISSIONS ---
# Grant the required Storage Object Admin role to the Service Account on the GCS bucket
print(f"\n--- FIXING PERMISSIONS ---")
print(f"Granting 'Storage Object Admin' role to {SERVICE_AGENT} on bucket {BUCKET}")

!gcloud storage buckets add-iam-policy-binding {BUCKET} \
--member=serviceAccount:{SERVICE_AGENT} \
--role=roles/storage.objectAdmin \
--project={PROJECT_ID}

# --- REDEPLOYMENT (EXPECTED TO SUCCEED) ---
AGENT_DISPLAY_NAME_SUCCESS = "adk-agent-success"

print("\n--- REDEPLOYMENT (EXPECTED TO SUCCEED) ---")
!adk deploy agent_engine \
    --project={PROJECT_ID} \
    --region={LOCATION} \
    --staging_bucket={BUCKET} \
    --display_name={AGENT_DISPLAY_NAME_SUCCESS} \
    .


--- FIXING PERMISSIONS ---
Granting 'Storage Object Admin' role to service-1083677030545@gcp-sa-aiplatform-customcode.iam.gserviceaccount.com on bucket gs://data-vpc-sc-demo-agent-artifacts
ERROR: (gcloud.storage.buckets.add-iam-policy-binding) HTTPError 400: Service account service-1083677030545@gcp-sa-aiplatform-customcode.iam.gserviceaccount.com does not exist.

--- REDEPLOYMENT (EXPECTED TO SUCCEED) ---
Copying agent source code...
Copying agent source code complete.
Initializing Vertex AI...
Resolving files and dependencies...
Vertex AI initialized.
Created /tmp/agent_engine_deploy_src/20251002_204138/agent_engine_app.py
Files and dependencies resolved
Running `absolufy-imports /tmp/agent_engine_deploy_src/20251002_204138/ai-agent/main.py`
Running `absolufy-imports /tmp/agent_engine_deploy_src/20251002_204138/ai-agent/deployment/deploy.py`
Running `absolufy-imports /tmp/agent_engine_deploy_src/20251002_204138/ai-agent/modelarmor/agent.py`
Running `absolufy-imports /tmp/agent_engi

# 8. Cleanup

Remove the local cloned directories.

In [ ]:
# Navigate back to the content directory and clean up
%cd /content/
!rm -rf ai-agent ai-agent-deploy-ae